### Get access token
[Documentation](https://documentation.dataspace.copernicus.eu/APIs/Token.html)




### OAuth Setup
[Docs](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html#python), [Credentials](https://shapps.dataspace.copernicus.eu/dashboard/#/account/settings)



In [15]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import json
import pathlib
from osgeo import gdal
from copernicus import Client, get_labels, get_sentinel2_product_ids, download_sentinel2_products
import building_dection as bd

credentials = json.load(open('credentials.json'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
client = Client(credentials['username'], credentials['password'])

In [17]:
# aoi = "POLYGON((4.220581 50.958859,4.521264 50.953236,4.545977 50.906064,4.541858 50.802029,4.489685 50.763825,4.23843 50.767734,4.192435 50.806369,4.189689 50.907363,4.220581 50.958859))"

# results = client.search("2020-01-01", "2020-01-02", "SENTINEL-2", aoi)

In [18]:
sn7_path = pathlib.Path('D:\snap_data\sn7')
train_path = sn7_path / 'SN7_buildings_train' / 'train'
sentinel_path = sn7_path / 'sentinel-2'
vector_label_dir = sn7_path / 'vector_labels'

In [19]:
labels = get_labels(train_path)[:20]

# create dir if not exist
if not vector_label_dir.exists():
    vector_label_dir.mkdir()

# copy labels files to vector_label_dir
for label in labels:
    shutil.copy(label, vector_label_dir / label.name)

labels = [ vector_label_dir / label.name for label in labels]

product_ids = get_sentinel2_product_ids(client, labels)

L15-0331E-1257N_1327_3160_13: POLYGON((-121.68466515096087 37.9615752659464, -121.64058492652846 37.9615752659464, -121.64058492652846 37.99617689154867, -121.68466515096087 37.99617689154867, -121.68466515096087 37.9615752659464))
Found S2B_MSIL1C_20200110T185739_N0500_R113_T10SFH_20230426T213902.SAFE
L15-0357E-1223N_1429_3296_13: POLYGON((-117.20231626982802 33.100719225768806, -117.15811895229255 33.100719225768806, -117.15811895229255 33.13760723935068, -117.20231626982802 33.13760723935068, -117.20231626982802 33.100719225768806))
Found S2A_MSIL1C_20200109T183731_N0500_R027_T11SMS_20230428T041538.SAFE
L15-0358E-1220N_1433_3310_13: POLYGON((-117.02648733393058 32.58380527888377, -116.9824193392965 32.58380527888377, -116.9824193392965 32.62111062364426, -117.02648733393058 32.62111062364426, -117.02648733393058 32.58380527888377))
Found S2A_MSIL1C_20200129T183621_N0208_R027_T11SNS_20200129T215547.SAFE
L15-0361E-1300N_1446_2989_13: POLYGON((-116.4551329875268 43.643957292863114, -11

In [33]:
labels[1]

WindowsPath('D:/snap_data/sn7/SN7_buildings_train/train/L15-0357E-1223N_1429_3296_13/labels/global_monthly_2020_01_mosaic_L15-0357E-1223N_1429_3296_13_Buildings.geojson')

In [20]:
downloaded_products = download_sentinel2_products(client, product_ids, sentinel_path)

File D:\snap_data\sn7\sentinel-2\L15-0331E-1257N_1327_3160_13.zip already exists
File D:\snap_data\sn7\sentinel-2\L15-0357E-1223N_1429_3296_13.zip already exists
File D:\snap_data\sn7\sentinel-2\L15-0358E-1220N_1433_3310_13.zip already exists
File D:\snap_data\sn7\sentinel-2\L15-0361E-1300N_1446_2989_13.zip already exists
File D:\snap_data\sn7\sentinel-2\L15-0368E-1245N_1474_3210_13.zip already exists
File D:\snap_data\sn7\sentinel-2\L15-0434E-1218N_1736_3318_13.zip already exists
File D:\snap_data\sn7\sentinel-2\L15-0487E-1246N_1950_3207_13.zip already exists
File D:\snap_data\sn7\sentinel-2\L15-0566E-1185N_2265_3451_13.zip already exists
File D:\snap_data\sn7\sentinel-2\L15-0571E-1075N_2287_3888_13.zip already exists
File D:\snap_data\sn7\sentinel-2\L15-0586E-1127N_2345_3680_13.zip already exists
File D:\snap_data\sn7\sentinel-2\L15-0595E-1278N_2383_3079_13.zip already exists
File D:\snap_data\sn7\sentinel-2\L15-0632E-0892N_2528_4620_13.zip already exists
File D:\snap_data\sn7\sentin

In [21]:
# check if product names contain "MSIL2A"
import shutil
for label, product in downloaded_products.items():
    print(label, product['Name'])
    exctract_path = sentinel_path / label
    # check if extract path contains l2a product
    try:
        for elem  in os.listdir(exctract_path):
            if 'MSIL2A' in elem:
                print( f"found {elem}, deleting {exctract_path}")
                shutil.rmtree(exctract_path)
    except FileNotFoundError:
        pass

L15-0331E-1257N_1327_3160_13 S2B_MSIL1C_20200110T185739_N0500_R113_T10SFH_20230426T213902.SAFE
L15-0357E-1223N_1429_3296_13 S2A_MSIL1C_20200109T183731_N0500_R027_T11SMS_20230428T041538.SAFE
L15-0358E-1220N_1433_3310_13 S2A_MSIL1C_20200129T183621_N0208_R027_T11SNS_20200129T215547.SAFE
L15-0361E-1300N_1446_2989_13 S2B_MSIL1C_20200117T184709_N0208_R070_T11TNJ_20200117T203714.SAFE
L15-0368E-1245N_1474_3210_13 S2A_MSIL1C_20200106T182741_N0500_R127_T11SPA_20230426T210208.SAFE
L15-0434E-1218N_1736_3318_13 S2A_MSIL1C_20200111T173701_N0500_R055_T13SFR_20230428T081153.SAFE
L15-0487E-1246N_1950_3207_13 S2A_MSIL1C_20200112T170651_N0500_R069_T15SUA_20230629T201230.SAFE
L15-0566E-1185N_2265_3451_13 S2A_MSIL1C_20200121T160511_N0500_R054_T17RNL_20230426T201445.SAFE
L15-0571E-1075N_2287_3888_13 S2A_MSIL1C_20200105T154541_N0500_R111_T17PPL_20230421T075135.SAFE
L15-0586E-1127N_2345_3680_13 S2A_MSIL1C_20200115T154541_N0208_R111_T18QTE_20200115T190541.SAFE
L15-0595E-1278N_2383_3079_13 S2B_MSIL1C_20200126T1

In [22]:
import zipfile

for k,v in downloaded_products.items():
    extract_path = sentinel_path/k
    if not extract_path.exists():
        print(f'extracting {v["zip_path"]} to {extract_path}')
        with zipfile.ZipFile(v['zip_path'], 'r') as zip_ref:
            zip_ref.extractall(extract_path)
    else:
        print(f'{extract_path} already exists')
    v['base_path'] = extract_path
    v['SAFE_path'] = list(extract_path.glob('*.SAFE'))[0]

D:\snap_data\sn7\sentinel-2\L15-0331E-1257N_1327_3160_13 already exists
extracting D:\snap_data\sn7\sentinel-2\L15-0357E-1223N_1429_3296_13.zip to D:\snap_data\sn7\sentinel-2\L15-0357E-1223N_1429_3296_13
extracting D:\snap_data\sn7\sentinel-2\L15-0358E-1220N_1433_3310_13.zip to D:\snap_data\sn7\sentinel-2\L15-0358E-1220N_1433_3310_13
extracting D:\snap_data\sn7\sentinel-2\L15-0361E-1300N_1446_2989_13.zip to D:\snap_data\sn7\sentinel-2\L15-0361E-1300N_1446_2989_13
D:\snap_data\sn7\sentinel-2\L15-0368E-1245N_1474_3210_13 already exists
extracting D:\snap_data\sn7\sentinel-2\L15-0434E-1218N_1736_3318_13.zip to D:\snap_data\sn7\sentinel-2\L15-0434E-1218N_1736_3318_13
extracting D:\snap_data\sn7\sentinel-2\L15-0487E-1246N_1950_3207_13.zip to D:\snap_data\sn7\sentinel-2\L15-0487E-1246N_1950_3207_13
D:\snap_data\sn7\sentinel-2\L15-0566E-1185N_2265_3451_13 already exists
D:\snap_data\sn7\sentinel-2\L15-0571E-1075N_2287_3888_13 already exists
extracting D:\snap_data\sn7\sentinel-2\L15-0586E-112

In [23]:
for k,v in downloaded_products.items():
    # print all paths in dict
    print(k)
    v["raster_label_path"] = v['base_path'] / f"{k}_raster.tif"
    bd.convert_geojson_to_utm_raster(v['geojson_path'], v["raster_label_path"])
    

L15-0331E-1257N_1327_3160_13
L15-0357E-1223N_1429_3296_13
L15-0358E-1220N_1433_3310_13
L15-0361E-1300N_1446_2989_13
L15-0368E-1245N_1474_3210_13
L15-0434E-1218N_1736_3318_13
L15-0487E-1246N_1950_3207_13
L15-0566E-1185N_2265_3451_13
L15-0571E-1075N_2287_3888_13
L15-0586E-1127N_2345_3680_13
L15-0595E-1278N_2383_3079_13
L15-0632E-0892N_2528_4620_13
L15-0683E-1006N_2732_4164_13
L15-0760E-0887N_3041_4643_13
L15-0924E-1108N_3699_3757_13
L15-0977E-1187N_3911_3441_13
L15-1014E-1375N_4056_2688_13
L15-1015E-1062N_4061_3941_13
L15-1025E-1366N_4102_2726_13


In [24]:
downloaded_products

{'L15-0331E-1257N_1327_3160_13': {'@odata.mediaContentType': 'application/octet-stream',
  'Id': '085e9579-18af-4f27-b479-48cd70b8bb30',
  'Name': 'S2B_MSIL1C_20200110T185739_N0500_R113_T10SFH_20230426T213902.SAFE',
  'ContentType': 'application/octet-stream',
  'ContentLength': 834506244,
  'OriginDate': '2023-10-22T12:45:10.355Z',
  'PublicationDate': '2023-10-23T06:44:31.006Z',
  'ModificationDate': '2023-10-23T06:44:40.220Z',
  'Online': True,
  'EvictionDate': '',
  'S3Path': '/eodata/Sentinel-2/MSI/L1C_N0500/2020/01/10/S2B_MSIL1C_20200110T185739_N0500_R113_T10SFH_20230426T213902.SAFE',
  'Checksum': [{'Value': 'b8e2b89923250a0b00d48ce077400380',
    'Algorithm': 'MD5',
    'ChecksumDate': '2023-10-23T06:44:38.073846Z'},
   {'Value': '78c42da3a1475430ceb456216ed6e67d75313ee6b4c49f82aec942071929fb14',
    'Algorithm': 'BLAKE3',
    'ChecksumDate': '2023-10-23T06:44:39.929872Z'}],
  'ContentDate': {'Start': '2020-01-10T18:57:39.024Z',
   'End': '2020-01-10T18:57:39.024Z'},
  'Footpr

In [25]:
locs = {}
for k,v in downloaded_products.items():
    locs[k] = v['SAFE_path']
    break
    
modifications = [
    {
        "flag": "SEN2COR_MODIFY_LUT_AEROSOL",
        "value": "MARITIME",
        "info": "Look Up Table Aerosol set to MARITIME",
        "name": "LUT_AEROSOL_MARITIME",
    },
    # {
    #     "flag": "SEN2COR_MODIFY_LUT_SEASON",
    #     "value": "WINTER",
    #     "info": "Look Up Table Mid Latitude Season set to WINTER",
    #     "name": "LUT_SEASON_WINTER",
    # },
]


In [26]:
import sys
sys.path.append("../")
from l2a_analysis import L2A_Analysis

l2a_analysis = L2A_Analysis(report_name="space_net_7_10_2020_copernicus", base_input_dir=sentinel_path, base_output_dir=sn7_path/"reports", resolution=10)
l2a_analysis.set_locations(locs)
l2a_analysis.set_modifications(modifications)


In [27]:
for loc, product in downloaded_products.items():
    roi = bd.get_region_of_interest(product["raster_label_path"], product["SAFE_path"])

    
    sen2cor_roi = bd.transform_roi_for_sen2cor(roi)
    l2a_analysis.add_region_of_interest(loc, sen2cor_roi)
    break

label_extent: ulx: 615497.0	uly: 4206249.0	lrx: 619397.0	lry: 4202389.0
SAFE_extent: ulx:  600000	uly: 4300020	lrx: 709800	lry: 4190220


In [28]:
l2a_analysis.run_sen2cor()

Running sen2cor for L15-0331E-1257N_1327_3160_13: Reference
Running command:  C:\Users\tlaus\Documents\local\semester_thesis\Sen2Cor-02.11.00-win64\L2A_Process.bat D:\snap_data\sn7\sentinel-2\L15-0331E-1257N_1327_3160_13\S2B_MSIL1C_20200110T185739_N0500_R113_T10SFH_20230426T213902.SAFE --resolution 10 --GIP_L2A C:\Users\tlaus\Documents\sen2cor\2.11\cfg\L2A_GIPP_custom.xml --output_dir D:\snap_data\sn7\reports/space_net_7_10_2020_copernicus/L15-0331E-1257N_1327_3160_13/reference
Running sen2cor for L15-0331E-1257N_1327_3160_13: LUT_AEROSOL_MARITIME
Running command:  C:\Users\tlaus\Documents\local\semester_thesis\Sen2Cor-02.11.00-win64\L2A_Process.bat D:\snap_data\sn7\sentinel-2\L15-0331E-1257N_1327_3160_13\S2B_MSIL1C_20200110T185739_N0500_R113_T10SFH_20230426T213902.SAFE --resolution 10 --GIP_L2A C:\Users\tlaus\Documents\sen2cor\2.11\cfg\L2A_GIPP_custom.xml --output_dir D:\snap_data\sn7\reports/space_net_7_10_2020_copernicus/L15-0331E-1257N_1327_3160_13/LUT_AEROSOL_MARITIME
